<a href="https://colab.research.google.com/github/nautiism/fin_papers_done/blob/master/ActorCriticTradeBot/AC_trade_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as autograd
import torch.optim as optim
import numpy as np
import torch.nn.init as init
from torch.distributions import Categorical


import pandas as pd
import numpy as np

In [0]:
def weight_init(m):
    if isinstance(m, nn.Conv1d):
        init.normal_(m.weight.data)
        if m.bias is not None:
            init.normal_(m.bias.data)
    elif isinstance(m, nn.Conv2d):
        init.xavier_normal_(m.weight.data)
        if m.bias is not None:
            init.normal_(m.bias.data)
    elif isinstance(m, nn.Conv3d):
        init.xavier_normal_(m.weight.data)
        if m.bias is not None:
            init.normal_(m.bias.data)
    elif isinstance(m, nn.ConvTranspose1d):
        init.normal_(m.weight.data)
        if m.bias is not None:
            init.normal_(m.bias.data)
    elif isinstance(m, nn.ConvTranspose2d):
        init.xavier_normal_(m.weight.data)
        if m.bias is not None:
            init.normal_(m.bias.data)
    elif isinstance(m, nn.ConvTranspose3d):
        init.xavier_normal_(m.weight.data)
        if m.bias is not None:
            init.normal_(m.bias.data)
    elif isinstance(m, nn.BatchNorm1d):
        init.normal_(m.weight.data, mean=1, std=0.02)
        init.constant_(m.bias.data, 0)
    elif isinstance(m, nn.BatchNorm2d):
        init.normal_(m.weight.data, mean=1, std=0.02)
        init.constant_(m.bias.data, 0)
    elif isinstance(m, nn.BatchNorm3d):
        init.normal_(m.weight.data, mean=1, std=0.02)
        init.constant_(m.bias.data, 0)
    elif isinstance(m, nn.Linear):
        init.xavier_normal_(m.weight.data)
        init.normal_(m.bias.data)
    elif isinstance(m, nn.LSTM):
        for param in m.parameters():
            if len(param.shape) >= 2:
                init.orthogonal_(param.data)
            else:
                init.normal_(param.data)
    elif isinstance(m, nn.LSTMCell):
        for param in m.parameters():
            if len(param.shape) >= 2:
                init.orthogonal_(param.data)
            else:
                init.normal_(param.data)
    elif isinstance(m, nn.GRU):
        for param in m.parameters():
            if len(param.shape) >= 2:
                init.orthogonal_(param.data)
            else:
                init.normal_(param.data)
    elif isinstance(m, nn.GRUCell):
        for param in m.parameters():
            if len(param.shape) >= 2:
                init.orthogonal_(param.data)
            else:
                init.normal_(param.data)

In [0]:
class TradeEnv():
    def __init__(self,share_count,start_cash,sbin,rel,cus=0,brokerage=0.1):
        self.broker = brokerage
        self.inaction_penalty = 0
        self.so = sbin['Open'];self.sc = sbin['Close'];
        self.ro = rel['Open'];self.rc = rel['Close']
        
        self.startS = cus
        self.cus = cus
        self.ss = 1 
        self.epi_length = len(self.so)
        
        self.start_cash = max(int(np.random.normal(start_cash['mu'],start_cash['sig'])),0)

        self.state = {}
        self.state['sbin_share_count']= max(int(np.random.normal(share_count['sbin']['mu'],share_count['sbin']['sig'])),0)
        self.state['rel_share_count']= max(int(np.random.normal(share_count['rel']['mu'],share_count['rel']['sig'])),0)
        self.state['liquidity'] = self.start_cash 
        self.align()
        self.init_portfolio = self.state['portfolio_val']
        
        self.end = False
        
    def align(self):
        self.state['portfolio_val'] = self.state['sbin_share_count'] * self.sc[self.cus] + self.state['rel_share_count'] * self.rc[self.cus]
        self.state['sbin_open'] = self.so[self.cus]
        self.state['rel_open'] = self.ro[self.cus]
        self.state['sbin_ma_5'] = self.so[self.cus] if self.cus < 5 else self.so[self.cus-5:self.cus].mean() 
        self.state['rel_ma_5'] = self.ro[self.cus] if self.cus < 5 else self.ro[self.cus-5:self.cus].mean()
        
    def gain(self):
        return self.state['portfolio_val']-self.init_porfolio
    
    def t_remain(self):
        return self.epi_length-(self.cus-self.startS) 
        
    def step(self,a):
        quant = 1
        bonus = 0
        if self.cus > self.epi_length:
            if self.state['sbin_share_count'] and self.state['rel_share_count']:
                bonus = 1
            return self.state.values(),self.state['portfolio_val']+bonus+self.gain(),True
        
        if a==0:
            #buy sbin
            if quant*self.so[self.cus] > self.state['liquidity']:
                self.align()
                return self.state.values(),-1*self.t_remain()+self.gain()/2,True
            else:
                self.state['sbin_share_count'] += quant
                self.state['liquidity'] -= quant*self.so[self.cus]*(1+self.broker)
                self.align()
                return self.state.values(),self.inaction_penalty-1*self.t_remain()+self.gain(),False
        elif a==1:
            #sell sbin
            if quant > self.state['sbin_share_count']:
                self.align()
                return self.state.values(),-1*self.t_remain()+self.gain()/2,True
            else:
                self.state['sbin_share_count']-=quant
                self.state['liquidity'] += quant*self.so[self.cus]*(1-self.broker)
                self.align()
                return self.state.values(),self.inaction_penalty-1*self.t_remain()+self.gain(),False
        elif a==2:
            return self.state.values(),-1*self.inaction_penalty-1*self.t_remain()+self.gain(),False
        elif a==3:
            #buy rel
            if quant*self.ro[self.cus] > self.state['liquidity']:
                self.align()
                return self.state.values(),-1*self.t_remain()+self.gain()/2,True
            else:
                self.state['rel_share_count'] += quant
                self.state['liquidity'] -= quant*self.ro[self.cus]*(1+self.broker)
                self.align()
                return self.state.values(),self.inaction_penalty-1*self.t_remain()+self.gain(),False
        elif a==4:
            #sell sbin
            if quant > self.state['rel_share_count']:
                self.align()
                return self.state.values(),-1*self.t_remain()+self.gain()/2,True
            else:
                self.state['sbin_share_count']-=quant
                self.state['liquidity'] += quant*self.ro[self.cus]*(1-self.broker)
                self.align()
                return self.state.values(),self.inaction_penalty-1*self.t_remain()+self.gain(),False
        
    def reset(self):
        self.cus = self.ss
        self.epi_length = len(self.so)
        self.start_cash = max(int(np.random.normal(start_cash['mu'],start_cash['sig'])),0)
        self.state = {}
        self.state['sbin_share_count']= max(int(np.random.normal(share_count['sbin']['mu'],share_count['sbin']['sig'])),0)
        self.state['rel_share_count']= max(int(np.random.normal(share_count['rel']['mu'],share_count['rel']['sig'])),0)
        self.state['liquidity'] = self.start_cash 
        self.align()
        self.init_porfolio = self.state['portfolio_val']



In [0]:
class ActorCritic(nn.Module):
    def __init__(self):
        super(ActorCritic, self).__init__()
        
        self.input_layer = nn.Linear(8, 128)
        self.hidden_1 = nn.Linear(128, 128)
        self.hidden_2 = nn.Linear(32,31)
        self.hidden_state = torch.zeros(2,1,32).cuda()
        self.rnn = nn.GRU(128, 32, 2)
        
        self.action_layer = nn.Linear(31, 5)
        self.value_layer = nn.Linear(31, 1)
          
        self.logprobs = []
        self.state_values = []
        self.rewards = []
        self.apply(weight_init)
        
    def forward(self, x):
        x = torch.FloatTensor(list(x)).cuda()
        x = self.input_layer(x)
        x = torch.tanh(x)
        x = torch.tanh(self.hidden_1(x))
        x, self.hidden_state = self.rnn(x.view(1,-1,128), self.hidden_state.data)
        x = F.relu(self.hidden_2(x.squeeze()))
        action_scores = self.action_layer(x)
        state_value = self.value_layer(x)
        action_probs = F.softmax(action_scores, dim=-1)
        return action_probs,state_value 
    
    def calculateLoss(self, gamma=0.99):
        rewards = []
        dis_reward = 0
        for reward in self.rewards[::-1]:
            dis_reward = reward + gamma * dis_reward
            rewards.insert(0, dis_reward)
        
        rewards = torch.tensor(rewards).cuda()
        if rewards.shape[0]==1:
          rewards = (rewards - rewards.mean())
        else:
          rewards = (rewards - rewards.mean()) / (rewards.std(False))
        
        loss = torch.Tensor([0]).cuda()
        for logprob, value, reward in zip(self.logprobs, self.state_values, rewards):
            advantage = reward  - value
            action_loss = -logprob * advantage
            value_loss = F.smooth_l1_loss(value, reward)
            loss += (action_loss + value_loss)
        return loss
    
    def clearMemory(self):
        del self.logprobs[:]
        del self.state_values[:]
        del self.rewards[:]

In [0]:
sbi = pd.read_csv('./drive/My Drive/reinforce/sbi.csv')
rel = pd.read_csv('./drive/My Drive/reinforce/rel.csv')

share_count = {'sbin':{'mu':100,'sig':10},
               'rel':{'mu':100,'sig':10}}
start_cash = {'mu':1000,'sig':100}
TE = TradeEnv(share_count,start_cash,sbi,rel)

In [9]:
episodes = 100000
total_profits = 0
ac = ActorCritic().cuda()
opt = optim.Adam(ac.parameters(), lr=3e-4)

for j in range(episodes):
    TE.reset()
    opt.zero_grad()
    state = list(TE.state.values())
    for i in range(0,TE.epi_length  + 1):
        actions,state_value = ac(state)
        action_distribution = Categorical(actions)
        action = action_distribution.sample()
        ac.logprobs.append(action_distribution.log_prob(action))
        ac.state_values.append(state_value)
        state, reward, done = TE.step(action)
        ac.rewards.append(reward)
        if done:
            break
    loss = ac.calculateLoss()
    loss.backward()
    opt.step()
    ac.clearMemory()
total_profits += (TE.state['portfolio_val'] - TE.init_portfolio) /  TE.init_portfolio
    
print("Avg % profit per game: {}".format(total_profits / episodes))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.


Avg % profit per game: -1.1426390524650747e-06


In [0]:
torch.FloatTensor([2.00,3.43,5.434,45.334]).shape[0]